# Uittesten van strategie elementen in Pandas voor CryptoMaven bot. 

De bedoeling is dat de strategie ``DEM9 > SMA 21 && RSI21 > (SMA9:RSI21)`` signalen geeft om long en short te gaan. Deze strategie is een mogelijke verbetering op de strategie ``DEMA 9 > SMA 21 || RSI21 > 55`` die voor Bitcoin 281% en Ethereum 292% winst gaf. 

## Data in dataframe laden en controleren op juistheid

Data wordt dmv. een python3 functie gedownload van Binance.

In [161]:
import pandas as pd
import pandas_ta as ta
import numpy as np

# Load data
df = pd.read_csv("./BTCUSDT_1d.csv")

# Reading columns
df.columns

Index(['# openTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime',
       'qouteAssetVolume', 'numberOfTrades', 'takerBuyBaseAssetVolume',
       'takerBuyQuoteAssetVolume', 'ignore'],
      dtype='object')

In [162]:
# Aanpassen van unix time naar human time
df['# openTime']=(pd.to_datetime(df['# openTime'],unit='ms'))
df['closeTime']=(pd.to_datetime(df['closeTime'],unit='ms'))

In [163]:
# Watch last 10 entries
df[['# openTime', 'open', 'high', 'low', 'close', 'volume']].tail(10)

,# openTime,open,high,low,close,volume
490,2020-10-18,11360.31,11505.00,11346.22,11503.14,23284.041191
491,2020-10-19,11503.14,11823.99,11407.96,11751.47,47414.534692
492,2020-10-20,11751.46,12038.38,11677.59,11909.99,62134.750663
493,2020-10-21,11910.00,13217.68,11886.95,12780.96,114584.456767
494,2020-10-22,12780.75,13185.00,12678.08,12968.52,70038.824144
495,2020-10-23,12968.84,13027.69,12720.08,12923.07,50386.999841
496,2020-10-24,12923.06,13166.73,12870.00,13111.73,35952.209070
497,2020-10-25,13111.73,13350.00,12888.00,13028.83,38481.579504
498,2020-10-26,13029.64,13238.81,12765.00,13052.19,60951.672986
499,2020-10-27,13052.15,13170.21,13019.87,13157.15,17425.768811


## Alle kolommen afronden op 2 cijfers achter de komma

In [164]:
df.round(2)

,# openTime,open,high,low,close,volume,closeTime,qouteAssetVolume,numberOfTrades,takerBuyBaseAssetVolume,takerBuyQuoteAssetVolume,ignore
0,2019-06-16,8810.77,9333.00,8760.00,8953.33,63289.25,2019-06-16 23:59:59.999,5.742113e+08,552319,33804.18,3.067937e+08,0
1,2019-06-17,8953.00,9444.00,8950.00,9313.96,47895.49,2019-06-17 23:59:59.999,4.415673e+08,462160,24826.98,2.289622e+08,0
2,2019-06-18,9312.13,9336.36,8950.00,9081.55,51554.57,2019-06-18 23:59:59.999,4.702720e+08,449730,28057.05,2.559724e+08,0
3,2019-06-19,9081.97,9304.00,8960.00,9255.49,32147.71,2019-06-19 23:59:59.999,2.937481e+08,306202,17533.23,1.603036e+08,0
4,2019-06-20,9253.76,9590.00,9175.20,9517.12,34556.05,2019-06-20 23:59:59.999,3.236218e+08,350662,18032.19,1.688917e+08,0
...,...,...,...,...,...,...,...,...,...,...,...,...
495,2020-10-23,12968.84,13027.69,12720.08,12923.07,50387.00,2020-10-23 23:59:59.999,6.506802e+08,699313,24685.93,3.188277e+08,0
496,2020-10-24,12923.06,13166.73,12870.00,13111.73,35952.21,2020-10-24 23:59:59.999,4.682995e+08,536821,18382.74,2.394554e+08,0
497,2020-10-25,13111.73,13350.00,12888.00,13028.83,38481.58,2020-10-25 23:59:59.999,5.026720e+08,610498,17988.77,2.351159e+08,0
498,2020-10-26,13029.64,13238.81,12765.00,13052.19,60951.67,2020-10-26 23:59:59.999,7.943957e+08,818144,29609.99,3.859867e+08,0


Toevoegen van DEMA9 en SMA21 aan de dataframe

In [165]:
df.ta.dema(length=9, append=True)
df.ta.sma(length=21, append=True)
df.ta.rsi(length=21, append=True)

df.columns

Index(['# openTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime',
       'qouteAssetVolume', 'numberOfTrades', 'takerBuyBaseAssetVolume',
       'takerBuyQuoteAssetVolume', 'ignore', 'DEMA_9', 'SMA_21', 'RSI_21'],
      dtype='object')

In [166]:
df[['# openTime', 'open', 'high', 'low', 'close', 'volume', 'DEMA_9', 'SMA_21', 'RSI_21']].tail(10)

,# openTime,open,high,low,close,volume,DEMA_9,SMA_21,RSI_21
490,2020-10-18,11360.31,11505.00,11346.22,11503.14,23284.041191,11507.666392,11022.028571,58.728326
491,2020-10-19,11503.14,11823.99,11407.96,11751.47,47414.534692,11622.782719,11072.283333,61.997911
492,2020-10-20,11751.46,12038.38,11677.59,11909.99,62134.750663,11758.671859,11123.212381,63.914032
493,2020-10-21,11910.00,13217.68,11886.95,12780.96,114584.456767,12165.242835,11218.658571,72.045599
494,2020-10-22,12780.75,13185.00,12678.08,12968.52,70038.824144,12517.598546,11330.534286,73.400880
495,2020-10-23,12968.84,13027.69,12720.08,12923.07,50386.999841,12744.781059,11442.566190,72.506466
496,2020-10-24,12923.06,13166.73,12870.00,13111.73,35952.209070,12965.227025,11564.931429,73.892998
497,2020-10-25,13111.73,13350.00,12888.00,13028.83,38481.579504,13082.328563,11677.417143,72.212735
498,2020-10-26,13029.64,13238.81,12765.00,13052.19,60951.672986,13163.543204,11785.035238,72.398437
499,2020-10-27,13052.15,13170.21,13019.87,13157.15,17425.768811,13248.852046,11906.820476,73.242086


Visuele controle van de waarden in Tradingview.

![Visuele controle](2020-10-27_11-50-51.png)

Cijfers kloppen! :-)

## Berekenen van SMA9 op RSI21
Idee is om de SMA9 los te laten op kolom RSI_21 en dan zien wat er gebeurd.

In Pandas-TA kun je niet de SMA direct verwijzen naar een specifieke kolom, dus moeten we een SMA maken op basis van andere informatie - zie o.a.: 

* https://stackoverflow.com/questions/40060842/moving-average-pandas
* https://www.datacamp.com/community/tutorials/moving-averages-in-pandas

Let's calculate SMA for a window size of 9, which means you will consider three values each time to calculate the moving average, and for every new value, the oldest value will be ignored.

To implement this, you will use pandas ``iloc`` function, since the ``rsi``
column is what you need, you will fix the position of that in the ``iloc`` function while the row will be a variable ``i`` which you will keep iterating until you reach the end of the dataframe.

In [167]:
df['RSI_SMA_14'] = df['RSI_21'].rolling(window=14).mean()


In [168]:
df.columns



Index(['# openTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime',
       'qouteAssetVolume', 'numberOfTrades', 'takerBuyBaseAssetVolume',
       'takerBuyQuoteAssetVolume', 'ignore', 'DEMA_9', 'SMA_21', 'RSI_21',
       'RSI_SMA_14'],
      dtype='object')

In [169]:
df[['# openTime','RSI_21', 'RSI_SMA_14']].tail(10)

,# openTime,RSI_21,RSI_SMA_14
490,2020-10-18,58.728326,55.604532
491,2020-10-19,61.997911,56.424702
492,2020-10-20,63.914032,57.585068
493,2020-10-21,72.045599,59.251151
494,2020-10-22,73.400880,60.736620
495,2020-10-23,72.506466,62.033085
496,2020-10-24,73.892998,63.200375
497,2020-10-25,72.212735,64.179574
498,2020-10-26,72.398437,65.032189
499,2020-10-27,73.242086,66.085075


## Kolommen controleren op long (1) of short (0) condities

Elke vergelijking krijgt een uitkomst mee:

* Als DEMA9 > SMA21 = 1
* Als RSI21 > RSI_MA_14 = 1

Als waarde is 0, dan is dit een SHORT conditie.  
Als waarde is 1, dan is dit een NEUTRAAL conditie ( = huidige positie houden en opletten).  
Als waarde is 2, dan is dit een LONG conditie.  



In [170]:
# SMA check
df['MA_pos'] = np.where(df['DEMA_9']>df['SMA_21'], 1 , 0)
# MFI check
df['RSI_pos'] = np.where(df['RSI_21']>df['RSI_SMA_14'], 1 , 0)

df[['# openTime','close','DEMA_9', 'SMA_21', 'RSI_21','RSI_SMA_14','MA_pos','RSI_pos']].tail(50)

,# openTime,close,DEMA_9,SMA_21,RSI_21,RSI_SMA_14,MA_pos,RSI_pos
450,2020-09-08,10126.65,10164.433672,11213.476667,40.990462,49.166618,0,0
451,2020-09-09,10219.20,10114.437431,11140.362857,42.095814,48.291625,0,0
452,2020-09-10,10336.87,10128.991539,11068.140000,43.508482,47.644181,0,0
453,2020-09-11,10387.89,10164.988506,11013.690000,44.129037,46.875534,0,0
454,2020-09-12,10440.92,10216.033425,10955.497619,44.790915,46.215690,0,0
455,2020-09-13,10332.83,10218.785236,10892.864286,43.683371,45.272885,0,0
456,2020-09-14,10671.77,10347.126586,10841.605714,47.923145,44.698406,0,1
457,2020-09-15,10785.31,10483.125186,10816.219524,49.266577,44.001018,0,1
458,2020-09-16,10954.01,10642.983683,10792.056667,51.229414,43.971505,0,1
459,2020-09-17,10939.99,10752.686974,10773.466667,51.057041,44.796893,0,1


Nu een kolom voor de telling van de posities en een kolom die obv. deze telling een advies geeft.

In [171]:
# Posities tellen
df['POS_cnt'] = df['MA_pos'] + df['RSI_pos']
df[['MA_pos','RSI_pos','POS_cnt']].tail(50)

,MA_pos,RSI_pos,POS_cnt
450,0,0,0
451,0,0,0
452,0,0,0
453,0,0,0
454,0,0,0
455,0,0,0
456,0,1,1
457,0,1,1
458,0,1,1
459,0,1,1


In [172]:
# Advies geven
def f(row):
    if row['POS_cnt'] == 1:
        val = 'NEUTRAAL'
    elif row['POS_cnt'] == 2:
        val = 'LONG'
    else:
        val = 'SHORT'
    return val

#create new column 'POS-adv' using the function above
df['POS_adv'] = df.apply(f, axis=1)

In [173]:
df[['# openTime','close','MA_pos','RSI_pos','POS_cnt','POS_adv']].tail(50)

,# openTime,close,MA_pos,RSI_pos,POS_cnt,POS_adv
450,2020-09-08,10126.65,0,0,0,SHORT
451,2020-09-09,10219.20,0,0,0,SHORT
452,2020-09-10,10336.87,0,0,0,SHORT
453,2020-09-11,10387.89,0,0,0,SHORT
454,2020-09-12,10440.92,0,0,0,SHORT
455,2020-09-13,10332.83,0,0,0,SHORT
456,2020-09-14,10671.77,0,1,1,NEUTRAAL
457,2020-09-15,10785.31,0,1,1,NEUTRAAL
458,2020-09-16,10954.01,0,1,1,NEUTRAAL
459,2020-09-17,10939.99,0,1,1,NEUTRAAL


Print nu alleen de waarde van de positie die ingenomen dient te worden van de laatste volle dag = 498 (regel 499 is altijd van de lopende dag en derhalve nog niet afgesloten).

In [174]:
print (df[['# openTime','POS_adv']].iloc[-2])

# openTime    2020-10-26 00:00:00
POS_adv                      LONG
Name: 498, dtype: object


In [175]:
advice = df.iloc[-2]['POS_adv']
date = df.iloc[-2]['# openTime']
print(date)
print(advice)

2020-10-26 00:00:00
LONG


## Toevoegen van ATR indicator

In [176]:
df.ta.atr(length=14, append=True)

df.columns

Index(['# openTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime',
       'qouteAssetVolume', 'numberOfTrades', 'takerBuyBaseAssetVolume',
       'takerBuyQuoteAssetVolume', 'ignore', 'DEMA_9', 'SMA_21', 'RSI_21',
       'RSI_SMA_14', 'MA_pos', 'RSI_pos', 'POS_cnt', 'POS_adv', 'ATR_14'],
      dtype='object')

In [177]:
df[['# openTime', 'ATR_14']].tail(5)

,# openTime,ATR_14
495,2020-10-23,387.472751
496,2020-10-24,380.991126
497,2020-10-25,386.777474
498,2020-10-26,392.994083
499,2020-10-27,375.661648


ATR bedragen kloppen met de Tradingview chart!

In [178]:
# Adding ATR and SL/TP1 levels
df['Close+atr']=df['close'] + df['ATR_14']
df['Close-atr']=df['close'] - df['ATR_14']
df[['# openTime','close','ATR_14','Close+atr','Close-atr']].tail(10)

,# openTime,close,ATR_14,Close+atr,Close-atr
490,2020-10-18,11503.14,300.123182,11803.263182,11203.016818
491,2020-10-19,11751.47,308.402240,12059.872240,11443.067760
492,2020-10-20,11909.99,312.144223,12222.134223,11597.845777
493,2020-10-21,12780.96,384.900350,13165.860350,12396.059650
494,2020-10-22,12968.52,393.616039,13362.136039,12574.903961
495,2020-10-23,12923.07,387.472751,13310.542751,12535.597249
496,2020-10-24,13111.73,380.991126,13492.721126,12730.738874
497,2020-10-25,13028.83,386.777474,13415.607474,12642.052526
498,2020-10-26,13052.19,392.994083,13445.184083,12659.195917
499,2020-10-27,13157.15,375.661648,13532.811648,12781.488352


# Alle kolommen

In [179]:
df.columns

Index(['# openTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime',
       'qouteAssetVolume', 'numberOfTrades', 'takerBuyBaseAssetVolume',
       'takerBuyQuoteAssetVolume', 'ignore', 'DEMA_9', 'SMA_21', 'RSI_21',
       'RSI_SMA_14', 'MA_pos', 'RSI_pos', 'POS_cnt', 'POS_adv', 'ATR_14',
       'Close+atr', 'Close-atr'],
      dtype='object')

In [180]:
df[['# openTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime',
       'qouteAssetVolume', 'numberOfTrades', 'takerBuyBaseAssetVolume',
       'takerBuyQuoteAssetVolume', 'ignore', 'DEMA_9', 'SMA_21', 'RSI_21',
       'RSI_SMA_14', 'MA_pos', 'RSI_pos', 'POS_cnt', 'POS_adv',
       'ATR_14', 'Close+atr', 'Close-atr']]

,# openTime,open,high,low,close,volume,closeTime,qouteAssetVolume,numberOfTrades,takerBuyBaseAssetVolume,...,SMA_21,RSI_21,RSI_SMA_14,MA_pos,RSI_pos,POS_cnt,POS_adv,ATR_14,Close+atr,Close-atr
0,2019-06-16,8810.77,9333.00,8760.00,8953.33,63289.251219,2019-06-16 23:59:59.999,5.742113e+08,552319,33804.182046,...,NaN,NaN,NaN,0,0,0,SHORT,573.000000,9526.330000,8380.330000
1,2019-06-17,8953.00,9444.00,8950.00,9313.96,47895.485374,2019-06-17 23:59:59.999,4.415673e+08,462160,24826.976850,...,NaN,100.000000,NaN,0,0,0,SHORT,532.037037,9845.997037,8781.922963
2,2019-06-18,9312.13,9336.36,8950.00,9081.55,51554.569401,2019-06-18 23:59:59.999,4.702720e+08,449730,28057.047335,...,NaN,59.641733,NaN,0,0,0,SHORT,479.838318,9561.388318,8601.711682
3,2019-06-19,9081.97,9304.00,8960.00,9255.49,32147.706495,2019-06-19 23:59:59.999,2.937481e+08,306202,17533.233016,...,NaN,69.359429,NaN,0,0,0,SHORT,442.015858,9697.505858,8813.474142
4,2019-06-20,9253.76,9590.00,9175.20,9517.12,34556.053982,2019-06-20 23:59:59.999,3.236218e+08,350662,18032.190714,...,NaN,77.801258,NaN,0,0,0,SHORT,435.737601,9952.857601,9081.382399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2020-10-23,12968.84,13027.69,12720.08,12923.07,50386.999841,2020-10-23 23:59:59.999,6.506802e+08,699313,24685.925495,...,11442.566190,72.506466,62.033085,1,1,2,LONG,387.472751,13310.542751,12535.597249
496,2020-10-24,12923.06,13166.73,12870.00,13111.73,35952.209070,2020-10-24 23:59:59.999,4.682995e+08,536821,18382.744579,...,11564.931429,73.892998,63.200375,1,1,2,LONG,380.991126,13492.721126,12730.738874
497,2020-10-25,13111.73,13350.00,12888.00,13028.83,38481.579504,2020-10-25 23:59:59.999,5.026720e+08,610498,17988.767100,...,11677.417143,72.212735,64.179574,1,1,2,LONG,386.777474,13415.607474,12642.052526
498,2020-10-26,13029.64,13238.81,12765.00,13052.19,60951.672986,2020-10-26 23:59:59.999,7.943957e+08,818144,29609.994698,...,11785.035238,72.398437,65.032189,1,1,2,LONG,392.994083,13445.184083,12659.195917
